# Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Installs 

In [ ]:
!pip install -r requirements.txt

# Imports

In [ ]:
import json
import os

import cv2
from skimage import filters
from einops import rearrange
import statistics
import seaborn as sns; sns.set_theme()

import random
import traceback
import nibabel as nib
import scipy 

import numpy as np
from numpy import save
import matplotlib.pyplot as plt
import time
from datetime import datetime

import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras import layers
from plot_keras_history import plot_history

from sklearn.model_selection import ParameterGrid
from sklearn import metrics

from scripts.evalresults import *
from scripts.utils import *

# Autoencoders

## Dense Convolutional Autoencoder

In [ ]:
#-- Model implementation : Dense Convolutional Autoencoder

def DCAE():
  
    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels)) 

    x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(input_img)
    x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    x = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)
    
    x = layers.Flatten()(x)
    encoded = layers.Dense(intermediate_dim, activation=layers.LeakyReLU())(x)

    #-- BOTTELNECK SIZE : 512 
    
    x = layers.Dense(16 * 16 * 16, activation=layers.LeakyReLU())(encoded)
    x = layers.Reshape((16, 16, 16))(x)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(x)    
    x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    
    decoded = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)
    
    model = tf.keras.Model(input_img, decoded)

    return model

    
#-- Configure the hyperparameters

model_name = 'Dense Convolutional Autoencoder'
numEpochs = 50
learning_rate = 0.00001
image_size = 256
num_channels = 1
batch_size = 1
intermediate_dim = 512


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
      
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
 

#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
          
calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar() 

model = DCAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {intermediate_dim}: Bottelneck size\n')
print(parameters)

           
#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories 

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test  

print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                      
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                         
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()


#-- End

print('\nEnd !\n')

## Spatial Convolutional Autoencoder

In [ ]:
#-- Model implementation : Spatial Convolutional Autoencoder

def SCAE():
  
    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels)) 

    x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(input_img)
    x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    encoded = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)
   
    #-- BOTTELNECK SIZE : (16, 16, 16)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(encoded)    
    x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    
    decoded = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)
    
    model = tf.keras.Model(input_img, decoded)

    return model

    
#-- Configure the hyperparameters

model_name = 'Spatial Convolutional Autoencoder'
numEpochs = 50
learning_rate = 0.00001
image_size = 256
num_channels = 1
batch_size = 1
intermediate_dim = (16, 16, 16)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'    
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
     
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
      

#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
          
calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar() 

model = SCAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {intermediate_dim}: Bottelneck size\n')
print(parameters)

         
#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories   

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test  
    
print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                    
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                    
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()

      
#-- End

print('\nEnd !\n')

# Latent Variable Models

## Variational Autoencoder

In [ ]:
#-- Model implementation : Variational Autoencoder

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
        

def VAE():

  inputs = tf.keras.Input(shape=(image_size, image_size, num_channels))
  
  x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(inputs)
  x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
  x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
  x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
  x = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)   
  x = layers.Flatten()(x)
  encoded = layers.Dense(intermediate_dim, activation=layers.LeakyReLU())(x)   
  
  z_mean = layers.Dense(latent_dim, name="z_mean")(encoded)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(encoded)
  z = Sampling()([z_mean, z_log_var])
  
  x = layers.Dense(16 * 16 * 16, activation=layers.LeakyReLU())(z)
  x = layers.Reshape((16, 16, 16))(x)
  x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(x)    
  x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
  x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  decoder_outputs = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)
  
  model = tf.keras.Model(inputs, decoder_outputs)
  
  return model


#-- Configure the hyperparameters

model_name = 'Variational Autoencoder'
numEpochs = 50
learning_rate = 0.00001
rate = 0.  
image_size = 256
num_channels = 1
batch_size = 1
latent_dim = 512
intermediate_dim = 2048


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy' 
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'    
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
     
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')

      
#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
           
calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar() 

model = VAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {intermediate_dim}: Intermediate dim\n - {latent_dim}: Bottelneck size\n')
print(parameters)

  
#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories 

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test

print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                      
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                       
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()

      
#-- End

print('\nEnd !\n')

## VQ-VAE

In [ ]:
#-- Model implementation : Vector Quantizer Variational Autoencoder

class VectorQuantizer(layers.Layer):
    def __init__(self, num_embeddings, embedding_dim, beta=0.25, **kwargs):
        super().__init__(**kwargs)
        self.embedding_dim = embedding_dim
        self.num_embeddings = num_embeddings
        self.beta = (
            beta  # This parameter is best kept between [0.25, 2] as per the paper.
        )

        # Initialize the embeddings which we will quantize.
        w_init = tf.random_uniform_initializer()
        self.embeddings = tf.Variable(
            initial_value=w_init(
                shape=(self.embedding_dim, self.num_embeddings), dtype="float32"
            ),
            trainable=True,
            name="embeddings_vqvae",
        )

    def call(self, x):
        # Calculate the input shape of the inputs and
        # then flatten the inputs keeping `embedding_dim` intact.
        input_shape = tf.shape(x)
        flattened = tf.reshape(x, [-1, self.embedding_dim])

        # Quantization.
        encoding_indices = self.get_code_indices(flattened)
        encodings = tf.one_hot(encoding_indices, self.num_embeddings)
        quantized = tf.matmul(encodings, self.embeddings, transpose_b=True)
        quantized = tf.reshape(quantized, input_shape)

        commitment_loss = self.beta * tf.reduce_mean(
            (tf.stop_gradient(quantized) - x) ** 2
        )
        codebook_loss = tf.reduce_mean((quantized - tf.stop_gradient(x)) ** 2)
        self.add_loss(commitment_loss + codebook_loss)

        # Straight-through estimator.
        quantized = x + tf.stop_gradient(quantized - x)
        return quantized

    def get_code_indices(self, flattened_inputs):
        # Calculate L2-normalized distance between the inputs and the codes.
        similarity = tf.matmul(flattened_inputs, self.embeddings)
        distances = (
            tf.reduce_sum(flattened_inputs ** 2, axis=1, keepdims=True)
            + tf.reduce_sum(self.embeddings ** 2, axis=0)
            - 2 * similarity
        )

        # Derive the indices for minimum distances.
        encoding_indices = tf.argmin(distances, axis=1)
        return encoding_indices
      
              
def VQVAE():

    inputs = tf.keras.Input(shape=(image_size, image_size, num_channels))
    
    x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(inputs)
    x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    encoder_outputs = layers.Conv2D(latent_dim , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)

    quantized_latents = VectorQuantizer(num_embeddings, latent_dim, name="vector_quantizer")(encoder_outputs)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(quantized_latents)    
    x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    decoder_outputs = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)

    model = tf.keras.Model(inputs, decoder_outputs)

    return model


#-- Configure the hyperparameters

model_name = 'Vector Quantizer Variational Autoencoder'
numEpochs = 50
learning_rate = 0.00001
rate = 0.  
image_size = 256
num_channels = 1
batch_size = 1
latent_dim = 16
num_embeddings = 512
intermediate_dim = 4096


#-- Configure training and testing Datasets

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
    
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')

      
#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)    

calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar()  

model = VQVAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {num_embeddings}: Bottelneck size\n - {intermediate_dim}: Intermediate dim\n')
print(parameters)


#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories 

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test  

print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                     
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                   
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()
  
   
#-- End

print('\nEnd !\n')

# Transformer based models

## B_TAE

In [ ]:
#-- Implement patch creation as a layer

class Patches(layers.Layer):

    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'patch_size': self.patch_size
        })
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID")
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


#-- Implement the patch encoding layer

class PatchEncoder(layers.Layer):

    def __init__(self, embed_shape):
        super(PatchEncoder, self).__init__()
        self.embed_shape = embed_shape
        self.position_embedding = layers.Embedding(
            input_dim = embed_shape[0],
            output_dim = embed_shape[1]
            )
        self.projection = layers.Dense(units=embed_shape[1])
        
    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'embed_shape': self.embed_shape,
          'position_embedding': self.position_embedding,
          'projection': self.projection
        })
      return config

    def call(self, patches):
        positions = tf.range(start=0, limit=self.embed_shape[0], delta=1)
        encoded = self.projection(patches) + self.position_embedding(positions)
        return encoded


#-- Implement image reconstruction from patches as a layer

class Images(layers.Layer):
  
    def __init__(self, image_size, patch_size, num_channels):
        super(Images, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_channels = num_channels

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'image_size': self.image_size,
            'patch_size': self.patch_size,
            'num_channels': self.num_channels
          })
        return config

    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        reconstructed = tf.reshape(patches, [batch_size, self.image_size, self.image_size, num_channels])
        rec_new = tf.nn.space_to_depth(reconstructed, self.patch_size) 
        image = tf.reshape(rec_new, [batch_size, self.image_size, self.image_size, self.num_channels])   
        return image


#-- Dense Layer    

class TruncatedDense(layers.Dense):
    def __init__(self, units, use_bias=True, initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=.02)):
        super().__init__(units, use_bias=use_bias, kernel_initializer=initializer)

        
#-- Mlp Head

class Mlp(layers.Layer):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=layers.Activation(tf.nn.gelu), drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = TruncatedDense(hidden_features)
        self.act = act_layer
        self.fc2 = TruncatedDense(out_features)
        self.drop = layers.Dropout(drop)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'fc1': self.fc1,
            'act': self.act,
            'fc2': self.fc2,
            'drop': self.drop
          })
        return config

    def call(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


#-- Model implementation : Basic Vision Transformer Autoencoder 

def transformer_autoencoder(encoded_patches):

    for _ in range(transformer_layers):
        query = key = encoded_patches
        attn_encoded_patches = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout = rate)(query=query, value=encoded_patches, key=key)   
        attn_encoded_patches = layers.Dropout(rate)(attn_encoded_patches) 
        encoded_patches += attn_encoded_patches  
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        encoded_patches += ffn_out   
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)

    positions = tf.range(start=0, limit = embed_shape[0], delta=1)
    pos_embed = layers.Embedding(
        input_dim = embed_shape[0],
        output_dim = embed_shape[1])(positions)
    target = encoded_patches

    for _ in range(transformer_layers):
        query_tgt = key_tgt = target + pos_embed    
        attn_target1 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=target, key=key_tgt)   
        attn_target1 = layers.Dropout(rate)(attn_target1)
        target += attn_target1
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        query_tgt = target + pos_embed
        attn_target2 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=encoded_patches + pos_embed, key=encoded_patches)
        attn_target2 = layers.Dropout(rate)(attn_target2)
        target += attn_target2
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        target +=ffn_out
        target = layers.LayerNormalization(epsilon=1e-6)(target)
    return target


#-- Build the B_TAE model

def TAE():

    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels))
    
    patches = Patches(patch_size)(input_img)
    enc_patches = PatchEncoder(embed_shape)(patches)
    decoded_patches = transformer_autoencoder(enc_patches)
    decoded_patches = layers.Dense(units=patch_size*patch_size*num_channels)(decoded_patches)  
    reconstructed = Images(image_size, patch_size, num_channels)(decoded_patches)

    model = tf.keras.Model(input_img, reconstructed)

    return model
 
 
#-- Configure the hyperparameters

model_name = 'BASIC Transformer'
numEpochs = 50
learning_rate = 0.00001
rate = 0.1  
image_size = 256
num_channels = 1
batch_size = 4
filters = 96


#-- Transformer parameters variation

param_grid = {
              'transformer_layers': [8],
              'patch_size' : [16],
              'num_heads' : [4]
             }
             
            
PARAMS = ParameterGrid(param_grid)
list_PARAMS = list(PARAMS)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Train, Test and Evaluate

for param in list_PARAMS:
   try:
   
        #-- Checkpoints dir

        date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
        ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
        os.makedirs(ckpts_dir)
        
        ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
        params_path = os.path.join(ckpts_dir, 'Parameters.txt')
        results_path = os.path.join(ckpts_dir, 'Results.txt')
        fig_path = os.path.join(ckpts_dir, 'History_plot.png')
        dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
        predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
        residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
        residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
      

        #-- Configure the parameters

        transformer_layers = param['transformer_layers']
        patch_size = param['patch_size']
        num_heads = param['num_heads']
        input_resolution = image_size // patch_size
        num_patches = input_resolution ** 2
        embed_shape = (num_patches, filters)


        #-- Configure the training

        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
            
        calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
        tqdm_callback = tfa.callbacks.TQDMProgressBar() 

        model = TAE()
        model.summary()        
        model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


        #-- Print & Write model Parameters

        parameters = (f'\nSelected model "{model_name}" with :\n - {num_heads}: Heads,\n - {patch_size}: Patch size,\n - {transformer_layers}: Transformer Layer,\n - ({embed_shape[0]}, {embed_shape[1]}): Embedding Shape,\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n')
        print(parameters)
     
        
        #-- Train 

        print('\nTrain =>\n')
        history = model.fit(x = data_gen,
                            steps_per_epoch = training_steps,
                            validation_data = val_gen,
                            validation_steps = validation_steps,
                            verbose = 0,
                            epochs = numEpochs,
                            callbacks = [calbks, tqdm_callback]
                            )
        
                        
        #-- Get training and test loss histories 

        plot_history(history, path=fig_path)
        plt.close()
        time.sleep(2)
        
                
        #-- Test 

        print('\nTest ===>\n')
        my_test = np.load(test_path)
        brainmask = np.load(brainmask_path)
        x_prior = np.load(x_prior_path)
        my_labels = np.load(label_path)    
                
        healthy_test = np.load(test_healthy_path)
        steps = healthy_test.shape[0]
                
        score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])
        score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM: {score[2]},\n - Test MS_SSIMLoss: {score[3]},\n')
        print(score_out)
        
                
        #-- Predict

        print('\nPredict =====>\n')
        predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
        np.save(predicted_path, predicted)
        time.sleep(4)
        
        
        #-- Calculate, Post-process and Save Residuals

        print('\nCalculate, Post-process and Save Residuals =====>\n')     
        residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_BP_path, residual_BP)
        
        residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_path, residual)
        

        #-- Evaluation

        print('\nEvaluate =========>\n')        
        [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
        results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
        print(results)
      
                 
        plt.figure()
        hor_axis = [x for x in range(len_testset)]
        plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
        plt.ylabel('Dice Score')
        plt.xlabel('N° Samples')
        plt.title('Dice scores')
        plt.savefig(dice_plot_path)
        time.sleep(2)


        #-- Save

        print('\nSave Results and Parameters =============>\n')
        f = open(results_path, "w")
        f.write(results)       
        f.close()   
         
                        
        f = open(params_path, "w")
        f.write(parameters)
        f.write(score_out)
        f.close()
        
         
        #-- End

        print('\nEnd !\n')

   except:
        print('Error encountered !')
        tf.keras.backend.clear_session()
        continue

## DC_TAE

In [ ]:
#-- Implement patch creation as a layer

class Patches(layers.Layer):

    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'patch_size': self.patch_size
        })
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID")
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


#-- Implement the patch encoding layer

class PatchEncoder(layers.Layer):

    def __init__(self, embed_shape):
        super(PatchEncoder, self).__init__()
        self.embed_shape = embed_shape
        self.position_embedding = layers.Embedding(
            input_dim = embed_shape[0],
            output_dim = embed_shape[1]
            )
        self.projection = layers.Dense(units=embed_shape[1])
        
    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'embed_shape': self.embed_shape,
          'position_embedding': self.position_embedding,
          'projection': self.projection
        })
      return config

    def call(self, patches):
        positions = tf.range(start=0, limit=self.embed_shape[0], delta=1)
        encoded = self.projection(patches) + self.position_embedding(positions)
        return encoded


#-- Implement image reconstruction from patches as a layer

class Images(layers.Layer):
  
    def __init__(self, image_size, patch_size, num_channels):
        super(Images, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_channels = num_channels

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'image_size': self.image_size,
            'patch_size': self.patch_size,
            'num_channels': self.num_channels
          })
        return config

    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        reconstructed = tf.reshape(patches, [batch_size, self.image_size, self.image_size, num_channels])
        rec_new = tf.nn.space_to_depth(reconstructed, self.patch_size) 
        image = tf.reshape(rec_new, [batch_size, self.image_size, self.image_size, self.num_channels])   
        return image


#-- Dense Layer    

class TruncatedDense(layers.Dense):
    def __init__(self, units, use_bias=True, initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=.02)):
        super().__init__(units, use_bias=use_bias, kernel_initializer=initializer)


#-- Mlp Head

class Mlp(layers.Layer):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=layers.Activation(tf.nn.gelu), drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = TruncatedDense(hidden_features)
        self.act = act_layer
        self.fc2 = TruncatedDense(out_features)
        self.drop = layers.Dropout(drop)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'fc1': self.fc1,
            'act': self.act,
            'fc2': self.fc2,
            'drop': self.drop
          })
        return config

    def call(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


#-- Dense Convolutional Autoencoder

def DCAE(input_image):

    x = layers.Conv2D(32 , 3, activation=layers.LeakyReLU(), strides=2, padding="same")(input_image)
    x = layers.Conv2D(64 , 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    x = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)
    
    x = layers.Flatten()(x)
    encoded = layers.Dense(intermediate_dim, activation=layers.LeakyReLU())(x)

    #-- BOTTELNECK SIZE : 512 
    
    x = layers.Dense(16 * 16 * 16, activation=layers.LeakyReLU())(encoded)
    x = layers.Reshape((16, 16, 16))(x)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(x)    
    x = layers.Conv2DTranspose(128, 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    
    decoded = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)

    return decoded


#-- Model implementation : Dense Convolutional Autoencoder inside Transformer

def transformer_autoencoder(encoded_patches):

    for _ in range(transformer_layers):
        query = key = encoded_patches
        attn_encoded_patches = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout = rate)(query=query, value=encoded_patches, key=key)   
        attn_encoded_patches = layers.Dropout(rate)(attn_encoded_patches) 
        encoded_patches += attn_encoded_patches  
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        encoded_patches += ffn_out   
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    
    #-- Convolutional AutoEncoder
    encoded_img = Images(image_size, patch_size, num_channels)(encoded_patches)
    decoded_patches = DCAE(encoded_img)  
    encoded_patches = Patches(patch_size)(decoded_patches)
    encoded_patches = PatchEncoder(embed_shape)(encoded_patches)

    positions = tf.range(start=0, limit = embed_shape[0], delta=1)
    pos_embed = layers.Embedding(
        input_dim = embed_shape[0],
        output_dim = embed_shape[1])(positions)
    target = encoded_patches

    for _ in range(transformer_layers):
        query_tgt = key_tgt = target + pos_embed    
        attn_target1 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=target, key=key_tgt)   
        attn_target1 = layers.Dropout(rate)(attn_target1)
        target += attn_target1
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        query_tgt = target + pos_embed
        attn_target2 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=encoded_patches + pos_embed, key=encoded_patches)
        attn_target2 = layers.Dropout(rate)(attn_target2)
        target += attn_target2
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        target +=ffn_out
        target = layers.LayerNormalization(epsilon=1e-6)(target)
    return target


#-- Build the DC_TAE Model

def TAE():
  
    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels))

    patches = Patches(patch_size)(input_img)    
    enc_patches = PatchEncoder(embed_shape)(patches)

    decoded_patches = transformer_autoencoder(enc_patches)
  
    reconstructed = Images(image_size, patch_size, num_channels)(decoded_patches)

    model = tf.keras.Model(input_img, reconstructed)
    
    return model
 
    
#-- Configure the hyperparameters

model_name = 'DCAE Inside Transformer'
numEpochs = 50
learning_rate = 0.00001
rate = 0.1  
image_size = 256
num_channels = 1
batch_size = 1
intermediate_dim = 512


#-- Transformer parameters variation

param_grid = {
              'transformer_layers': [8],      
              'patch_size' : [16],
              'num_heads' : [4]
             }
             
            
PARAMS = ParameterGrid(param_grid)
list_PARAMS = list(PARAMS)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Train, Test and Evaluate

for param in list_PARAMS:
  try:

        #-- Checkpoints dir

        date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
        ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
        os.makedirs(ckpts_dir)
        
        ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
        params_path = os.path.join(ckpts_dir, 'Parameters.txt')
        results_path = os.path.join(ckpts_dir, 'Results.txt')
        fig_path = os.path.join(ckpts_dir, 'History_plot.png')
        dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
        predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
        residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
        residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
      

        #-- Configure the parameters

        transformer_layers = param['transformer_layers']
        patch_size = param['patch_size']
        num_heads = param['num_heads']
        input_resolution = image_size // patch_size
        num_patches = input_resolution ** 2
        embed_shape = (num_patches, patch_size*patch_size*num_channels)


        #-- Configure the training

        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
            
        calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
        tqdm_callback = tfa.callbacks.TQDMProgressBar() 

        model = TAE()
        model.summary()        
        model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])       
    

        #-- Print & Write model Parameters

        parameters = (f'\nSelected model "{model_name}" with :\n - {num_heads}: Heads,\n - {patch_size}: Patch size,\n - {transformer_layers}: Transformer Layer,\n - ({embed_shape[0]}, {embed_shape[1]}): Embedding Shape,\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n')
        print(parameters)
        
        
        #-- Train

        print('\nTrain =>\n')
        history = model.fit(x = data_gen,
                            steps_per_epoch = training_steps,
                            validation_data = val_gen,
                            validation_steps = validation_steps,
                            verbose = 0,
                            epochs = numEpochs,
                            callbacks = [calbks, tqdm_callback]
                            )
        
                        
        #-- Get training and test loss histories 

        plot_history(history, path=fig_path)
        plt.close()
        time.sleep(2)              
        
        
        #-- Test

        print('\nTest ===>\n')
        my_test = np.load(test_path)
        brainmask = np.load(brainmask_path)
        x_prior = np.load(x_prior_path)
        my_labels = np.load(label_path)              
               
        healthy_test = np.load(test_healthy_path)
        steps = healthy_test.shape[0]
                
        score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])
        score_out = (f'\nTest on healthy unseen data :\n - Test loss (MSE): {score[0]},\n - Test MAE : {score[1]},\n - Test SSIM: {score[2]},\n - Test MS_SSIMLoss: {score[3]},\n')
        print(score_out)
        
                
        #-- Predict

        print('\nPredict =====>\n')
        predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
        np.save(predicted_path, predicted)
        time.sleep(4)
        
        
        #-- Calculate, Post-process and Save Residuals

        print('\nCalculate, Post-process and Save Residuals =====>\n')     
        residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_BP_path, residual_BP)
        
        residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_path, residual)
        

        #-- Evaluation

        print('\nEvaluate =========>\n')        
        [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
        results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
        print(results)
                       
        plt.figure()
        hor_axis = [x for x in range(len_testset)]
        plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
        plt.ylabel('Dice Score')
        plt.xlabel('N° Samples')
        plt.title('Dice scores')
        plt.savefig(dice_plot_path)
        time.sleep(2)


        #-- Save

        print('\nSave Results and Parameters =============>\n')
        f = open(results_path, "w")
        f.write(results)       
        f.close()   
                    
        f = open(params_path, "w")
        f.write(parameters)
        f.write(score_out)
        f.close()       
         

        #-- End

        print('\nEnd of step !\n')
   

  except:
        print('Error encountered !')
        tf.keras.backend.clear_session()
        continue

## SC_TAE

In [ ]:
#-- Implement patch creation as a layer

class Patches(layers.Layer):

    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size
    
    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'patch_size': self.patch_size
        })
        return config

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID")
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


#-- Implement the patch encoding layer

class PatchEncoder(layers.Layer):

    def __init__(self, embed_shape):
        super(PatchEncoder, self).__init__()
        self.embed_shape = embed_shape
        self.position_embedding = layers.Embedding(
            input_dim = embed_shape[0],
            output_dim = embed_shape[1]
            )
        self.projection = layers.Dense(units=embed_shape[1])
        
    def get_config(self):
      config = super().get_config().copy()
      config.update({
          'embed_shape': self.embed_shape,
          'position_embedding': self.position_embedding,
          'projection': self.projection
        })
      return config

    def call(self, patches):
        positions = tf.range(start=0, limit=self.embed_shape[0], delta=1)
        encoded = self.projection(patches) + self.position_embedding(positions)
        return encoded


#-- Implement image reconstruction from patches as a layer

class Images(layers.Layer):
  
    def __init__(self, image_size, patch_size, num_channels):
        super(Images, self).__init__()
        self.image_size = image_size
        self.patch_size = patch_size
        self.num_channels = num_channels

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'image_size': self.image_size,
            'patch_size': self.patch_size,
            'num_channels': self.num_channels
          })
        return config

    def call(self, patches):
        batch_size = tf.shape(patches)[0]
        reconstructed = tf.reshape(patches, [batch_size, self.image_size, self.image_size, num_channels])
        rec_new = tf.nn.space_to_depth(reconstructed, self.patch_size) 
        image = tf.reshape(rec_new, [batch_size, self.image_size, self.image_size, self.num_channels])   
        return image


#-- Dense Layer  

class TruncatedDense(layers.Dense):
    def __init__(self, units, use_bias=True, initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=.02)):
        super().__init__(units, use_bias=use_bias, kernel_initializer=initializer)


#-- Mlp Head

class Mlp(layers.Layer):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=layers.Activation(tf.nn.gelu), drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = TruncatedDense(hidden_features)
        self.act = act_layer
        self.fc2 = TruncatedDense(out_features)
        self.drop = layers.Dropout(drop)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'fc1': self.fc1,
            'act': self.act,
            'fc2': self.fc2,
            'drop': self.drop
          })
        return config

    def call(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


#-- Spatial Convolutional Autoencoder

def SCAE(input_img):
  
    x = layers.Conv2D(32 , 3, activation=layers.LeakyReLU(), strides=2, padding="same")(input_img)
    x = layers.Conv2D(64 , 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 3, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    encoded = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)
   
    #-- BOTTELNECK SIZE : (16, 16, 16)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(encoded)    
    x = layers.Conv2DTranspose(128, 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 3, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    
    decoded = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)

    return decoded
  

#-- Model implementation : Spatial Convolutional Autoencoder inside Transformer

def transformer_autoencoder(encoded_patches):

    for _ in range(transformer_layers):
        query = key = encoded_patches
        attn_encoded_patches = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout = rate)(query=query, value=encoded_patches, key=key)   
        attn_encoded_patches = layers.Dropout(rate)(attn_encoded_patches) 
        encoded_patches += attn_encoded_patches  
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        encoded_patches += ffn_out   
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    
    #-- Convolutional AutoEncoder
    encoded_img = Images(image_size, patch_size, num_channels)(encoded_patches)
    decoded_patches = SCAE(encoded_img)  
    encoded_patches = Patches(patch_size)(decoded_patches)
    encoded_patches = PatchEncoder(embed_shape)(encoded_patches)

    positions = tf.range(start=0, limit = embed_shape[0], delta=1)
    pos_embed = layers.Embedding(
        input_dim = embed_shape[0],
        output_dim = embed_shape[1])(positions)
    target = encoded_patches

    for _ in range(transformer_layers):
        query_tgt = key_tgt = target + pos_embed    
        attn_target1 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=target, key=key_tgt)   
        attn_target1 = layers.Dropout(rate)(attn_target1)
        target += attn_target1
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        query_tgt = target + pos_embed
        attn_target2 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=encoded_patches + pos_embed, key=encoded_patches)
        attn_target2 = layers.Dropout(rate)(attn_target2)
        target += attn_target2
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        ffn_out = Mlp(
            in_features=embed_shape[1],
            hidden_features=4*embed_shape[1],
            drop=rate)(encoded_patches)
        target +=ffn_out
        target = layers.LayerNormalization(epsilon=1e-6)(target)
    return target


#-- Build the SC_TAE Model

def TAE():
  
    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels))

    patches = Patches(patch_size)(input_img)    
    enc_patches = PatchEncoder(embed_shape)(patches)

    decoded_patches = transformer_autoencoder(enc_patches)
  
    reconstructed = Images(image_size, patch_size, num_channels)(decoded_patches)

    model = tf.keras.Model(input_img, reconstructed)
    
    return model
 
    
#-- Configure the hyperparameters

model_name = 'SCAE Inside Transformer'
numEpochs = 50
learning_rate = 0.00001
rate = 0.1  
image_size = 256
num_channels = 1
batch_size = 1 


#-- Transformer parameters variation

param_grid = {
              'transformer_layers': [8],
              'patch_size' : [16],
              'num_heads' : [4]
             }
             
            
PARAMS = ParameterGrid(param_grid)
list_PARAMS = list(PARAMS)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Train, Test and Evaluate

for param in list_PARAMS:
   try:   

        #-- Checkpoints dir

        date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
        ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
        os.makedirs(ckpts_dir)

        ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
        params_path = os.path.join(ckpts_dir, 'Parameters.txt')
        results_path = os.path.join(ckpts_dir, 'Results.txt')
        fig_path = os.path.join(ckpts_dir, 'History_plot.png')
        dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
        predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
        residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
        residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
      

        #-- Configure the parameters

        transformer_layers = param['transformer_layers']
        patch_size = param['patch_size']
        num_heads = param['num_heads']
        input_resolution = image_size // patch_size
        num_patches = input_resolution ** 2
        embed_shape = (num_patches, patch_size*patch_size*num_channels)


        #-- Configure the training

        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
            
        calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
        tqdm_callback = tfa.callbacks.TQDMProgressBar() 

        model = TAE()
        model.summary()        
        model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


        #-- Print & Write model Parameters

        parameters = (f'\nSelected model "{model_name}" with :\n - {num_heads}: Heads,\n - {patch_size}: Patch size,\n - {transformer_layers}: Transformer Layer,\n - ({embed_shape[0]}, {embed_shape[1]}): Embedding Shape,\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n')
        print(parameters)
        
        
        #-- Train 

        print('\nTrain =>\n')
        history = model.fit(x = data_gen,
                            steps_per_epoch = training_steps,
                            validation_data = val_gen,
                            validation_steps = validation_steps,
                            verbose = 0,
                            epochs = numEpochs,
                            callbacks = [calbks, tqdm_callback]
                            )
        
                        
        #-- Get training and test loss histories

        plot_history(history, path=fig_path)
        plt.close()
        time.sleep(2)       
               
        
        #-- Test 

        print('\nTest ===>\n')
        my_test = np.load(test_path)
        brainmask = np.load(brainmask_path)
        x_prior = np.load(x_prior_path)
        my_labels = np.load(label_path)                     
        
        healthy_test = np.load(test_healthy_path)
        steps = healthy_test.shape[0]        
            
        score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])
        score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM: {score[2]},\n - Test MS_SSIMLoss: {score[3]},\n')
        print(score_out)
                      
                        
        #-- Predict

        print('\nPredict =====>\n')
        predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
        np.save(predicted_path, predicted)
        time.sleep(4)
        
        
        #-- Calculate, Post-process and Save Residuals

        print('\nCalculate, Post-process and Save Residuals =====>\n')     
        residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_BP_path, residual_BP)
        
        residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_path, residual)
        

        #-- Evaluation

        print('\nEvaluate =========>\n')        
        [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
        results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
        print(results)
                       
        plt.figure()
        hor_axis = [x for x in range(len_testset)]
        plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
        plt.ylabel('Dice Score')
        plt.xlabel('N° Samples')
        plt.title('Dice scores')
        plt.savefig(dice_plot_path)
        time.sleep(2)


        #-- Save

        print('\nSave Results and Parameters =============>\n')
        f = open(results_path, "w")
        f.write(results)       
        f.close()   
                              
        f = open(params_path, "w")
        f.write(parameters)
        f.write(score_out)
        f.close()
        
         
        #-- End

        print('\nEnd !\n')


   except:
        print('Error encountered !')
        tf.keras.backend.clear_session()
        continue

## H_TAE

In [ ]:
#-- Dense Layer   

class TruncatedDense(layers.Dense):
    def __init__(self, units, use_bias=True, initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=.02)):
        super().__init__(units, use_bias=use_bias, kernel_initializer=initializer)


#-- Patch Merging Layer

class PatchMerging(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim    #--- refer to the projection_dim (nC)
        self.reduction = TruncatedDense(2 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'reduction': self.reduction,
            'norm': self.norm
          })
        return config    

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim])
        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = tf.concat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = tf.reshape(x, [B, (H//2)*(W//2), 4 * self.dim])  # B H/2*W/2 4*C
        x = self.norm(x)
        x = self.reduction(x)

        return x


#-- Patch Expanding Layer  

class PatchExpanding(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = TruncatedDense(2 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'expand': self.expand,
            'norm': self.norm
          })
        return config

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        x = self.expand(x)
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim * 2])
        x = rearrange(x, 'b h w (p1 p2 c)-> b (h p1) (w p2) c', p1=2, p2=2, c=self.dim // 2)

        x = tf.reshape(x, [B, (H*2)*(W*2), self.dim//2])
        x = self.norm(x)

        return x


#-- Final Patch Expanding Layer

class FinalPatchExpand_X4(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = TruncatedDense(16 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'expand': self.expand,
            'norm': self.norm
          })
        return config

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        x = self.expand(x)
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim * 16])
        x = rearrange(x, 'b h w (p1 p2 c)-> b (h p1) (w p2) c', p1=4, p2=4, c=self.dim)
        x = tf.reshape(x, [B, (H*4)*(W*4), self.dim])

        x = self.norm(x)

        return x


#-- Mlp Head

class Mlp(layers.Layer):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=layers.Activation(tf.nn.gelu), drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = TruncatedDense(hidden_features)
        self.act = act_layer
        self.fc2 = TruncatedDense(out_features)
        self.drop = layers.Dropout(drop)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'fc1': self.fc1,
            'act': self.act,
            'fc2': self.fc2,
            'drop': self.drop
          })
        return config

    def call(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


#-- Model implementation : Hierarchical Transformer Autoencoder

def transformer_autoencoder(encoded_patches, embed_shape, input_resolution):
       
    for iter1 in range(transformer_layers//2):

      if iter1 != 0:
        encoded_patches = PatchMerging(input_resolution, embed_shape[1])(encoded_patches)
        input_resolution //= 2       
        embed_shape = (input_resolution*input_resolution, embed_shape[1]*2)
      
      
      for _ in range(2):
        query = key = encoded_patches
        #---Multi-head Attention Layer
        attn_encoded_patches = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query, value=encoded_patches, key=key)
            
        attn_encoded_patches = layers.Dropout(rate)(attn_encoded_patches)
        #---Skip Connection 1
        encoded_patches += attn_encoded_patches
        #---Layer Normalization 1
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        #---Feed-Forward Network
        ffn_out = Mlp(in_features=embed_shape[1], hidden_features=4*embed_shape[1], drop=rate)(encoded_patches)
        #---Skip Connection 
        encoded_patches += ffn_out
        #---Layer Normalization 2
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
      
      
    for iter2 in range(transformer_layers//2):

      if iter2 != 0:
        encoded_patches = PatchExpanding(input_resolution, embed_shape[1])(encoded_patches)
        input_resolution *= 2       
        embed_shape = (input_resolution*input_resolution, embed_shape[1]//2)
      
      
      target = encoded_patches  
      for _ in range(2):
        
        query_tgt = key_tgt = target
            
        #---First Multi-Head Attention
        attn_target1 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=target, key=key_tgt) 
                                    
        attn_target1 = layers.Dropout(rate)(attn_target1)
        #---Skip Connection 1
        target += attn_target1
        #---Layer Normalization 1
        target = layers.LayerNormalization(epsilon=1e-6)(target)

        query_tgt = target
            
        #---Second Multi-Head Attention
        attn_target2 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=encoded_patches, key=encoded_patches)
                                              
        attn_target2 = layers.Dropout(rate)(attn_target2)
        #---Skip Connection 2
        target += attn_target2
        #---Layer Normalization 2
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        #---Feed-Forward Network
        ffn_out = Mlp(in_features=embed_shape[1], hidden_features=4*embed_shape[1], drop=rate)(encoded_patches)
        #---Skip Connection 3
        target +=ffn_out
        #---Layer Normalization 3
        target = layers.LayerNormalization(epsilon=1e-6)(target)
      
      encoded_patches =  target

    return encoded_patches




"""# Build the H_TAE Model"""

def TAE():
   
  input_img = tf.keras.Input(shape=(image_size, image_size, num_channels))
  
  projection = layers.Conv2D(filters=filters, kernel_size=patch_size, strides=patch_size)(input_img)

  projection = layers.LayerNormalization(epsilon=1e-5)(projection) #-- si without

  B = tf.shape(projection)[0]
  enc_patches = tf.reshape(projection, [B, input_resolution*input_resolution, filters])

  decoded_patches = transformer_autoencoder(enc_patches, embed_shape, input_resolution)
  
  decoded_patches = FinalPatchExpand_X4(input_resolution, filters)(decoded_patches)
  
  B = tf.shape(decoded_patches)[0]
  decoded_patches = tf.reshape(decoded_patches, [B, image_size, image_size, filters])
  
  reconstructed = layers.Conv2D(filters=num_channels, kernel_size=1, use_bias=False)(decoded_patches)

  model = tf.keras.Model(input_img, reconstructed)
  
  return model
  
    
#-- Configure the hyperparameters

model_name = 'Hierarchical Transformer'
numEpochs = 50
learning_rate = 0.00001
rate = 0.  
image_size = 256
num_channels = 1
batch_size = 1


#-- Transformer parameters variation

param_grid = {
              'transformer_layers': [8],
              'patch_size' : [4],
              'num_heads' : [4],
              'filters' : [96]
             }
             
            
PARAMS = ParameterGrid(param_grid)
list_PARAMS = list(PARAMS)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Train, Test and Evaluate

for param in list_PARAMS:
   try:

        #-- Checkpoints dir

        date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
        ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
        os.makedirs(ckpts_dir)
        
        ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
        params_path = os.path.join(ckpts_dir, 'Parameters.txt')
        results_path = os.path.join(ckpts_dir, 'Results.txt')
        fig_path = os.path.join(ckpts_dir, 'History_plot.png')
        dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
        predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
        residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
        residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')


        #-- Configure the parameters

        transformer_layers = param['transformer_layers']
        patch_size = param['patch_size']
        num_heads = param['num_heads']
        filters = param['filters']
        input_resolution = image_size // patch_size
        num_patches = input_resolution ** 2
        embed_shape = (num_patches, filters)


        #-- Configure the training

        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)

        calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
        tqdm_callback = tfa.callbacks.TQDMProgressBar()

        model = TAE()
        model.summary()
        model.compile(optimizer=opt, loss='mse', metrics=['mae', SSIMLoss, MS_SSIMLoss])


        #-- Print & Write model Parameters

        parameters = (f'\nSelected model "{model_name}" with :\n - {num_heads}: Heads,\n - {patch_size}: Patch size,\n - {transformer_layers}: Transformer Layer,\n - ({embed_shape[0]}, {embed_shape[1]}): Embedding Shape,\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n')
        print(parameters)

        
        #-- Train

        print('\nTrain =>\n')
        history = model.fit(x = data_gen,
                            steps_per_epoch = training_steps,
                            validation_data = val_gen,
                            validation_steps = validation_steps,
                            verbose = 0,
                            epochs = numEpochs,
                            callbacks = [calbks, tqdm_callback]
                            )
        

        #-- Get training and test loss historie
        plot_history(history, path=fig_path)
        plt.close()
        time.sleep(2)               
        
        
        #-- Test 

        print('\nTest ===>\n')
        my_test = np.load(test_path)
        brainmask = np.load(brainmask_path)
        x_prior = np.load(x_prior_path)
        my_labels = np.load(label_path)              
        
        
        healthy_test = np.load(test_healthy_path)
        steps = healthy_test.shape[0]
                
        score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])
        score_out = (f'\nTest on healthy unseen data :\n - Test loss (MSE): {score[0]},\n - Test MAE : {score[1]},\n - Test SSIM: {score[2]},\n - Test MS_SSIMLoss: {score[3]},\n')
        print(score_out)
        
                
        #-- Predict

        print('\nPredict =====>\n')
        predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
        np.save(predicted_path, predicted)
        time.sleep(4)
        
        
        #-- Calculate, Post-process and Save Residuals

        print('\nCalculate, Post-process and Save Residuals =====>\n')     
        residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_BP_path, residual_BP)
        
        residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_path, residual)
        

        #-- Evaluation

        print('\nEvaluate =========>\n')        
        [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
        results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
        print(results)
                     
        plt.figure()
        hor_axis = [x for x in range(len_testset)]
        plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
        plt.ylabel('Dice Score')
        plt.xlabel('N° Samples')
        plt.title('Dice scores')
        plt.savefig(dice_plot_path)
        time.sleep(2)


        #-- Save

        print('\nSave Results and Parameters =============>\n')
        f = open(results_path, "w")
        f.write(results)       
        f.close()   
                
                
        f = open(params_path, "w")
        f.write(parameters)
        f.write(score_out)
        f.close()
        
        
        #-- End

        print('\nEnd !\n')


   except:
        print('Error encountered !')
        tf.keras.backend.clear_session()
        continue

## H_TAE_S

In [ ]:
#-- Dense Layer   

class TruncatedDense(layers.Dense):
    def __init__(self, units, use_bias=True, initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=.02)):
        super().__init__(units, use_bias=use_bias, kernel_initializer=initializer)


#-- Patch Merging Layer  

class PatchMerging(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim    #--- refer to the projection_dim (nC)
        self.reduction = TruncatedDense(2 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'reduction': self.reduction,
            'norm': self.norm
          })
        return config    

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim])
        x0 = x[:, 0::2, 0::2, :]  # B H/2 W/2 C
        x1 = x[:, 1::2, 0::2, :]  # B H/2 W/2 C
        x2 = x[:, 0::2, 1::2, :]  # B H/2 W/2 C
        x3 = x[:, 1::2, 1::2, :]  # B H/2 W/2 C
        x = tf.concat([x0, x1, x2, x3], -1)  # B H/2 W/2 4*C
        x = tf.reshape(x, [B, (H//2)*(W//2), 4 * self.dim])  # B H/2*W/2 4*C
        x = self.norm(x)
        x = self.reduction(x)

        return x


#-- Patch Expanding Layer 

class PatchExpanding(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = TruncatedDense(2 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'expand': self.expand,
            'norm': self.norm
          })
        return config

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        x = self.expand(x)
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim * 2])
        x = rearrange(x, 'b h w (p1 p2 c)-> b (h p1) (w p2) c', p1=2, p2=2, c=self.dim // 2)

        x = tf.reshape(x, [B, (H*2)*(W*2), self.dim//2])
        x = self.norm(x)

        return x


#-- Final Patch Expanding Layer 

class FinalPatchExpand_X4(layers.Layer):

    def __init__(self, input_resolution, dim):
        super().__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.expand = TruncatedDense(16 * dim, use_bias=False)
        self.norm = layers.LayerNormalization(epsilon=1e-5)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'input_resolution': self.input_resolution,
            'dim': self.dim,
            'expand': self.expand,
            'norm': self.norm
          })
        return config

    def call(self, x):
        """
        x: B, H*W, C
        """
        H = W = self.input_resolution
        x = self.expand(x)
        B = tf.shape(x)[0]

        x = tf.reshape(x, [B, H, W, self.dim * 16])
        x = rearrange(x, 'b h w (p1 p2 c)-> b (h p1) (w p2) c', p1=4, p2=4, c=self.dim)
        x = tf.reshape(x, [B, (H*4)*(W*4), self.dim])

        x = self.norm(x)

        return x


#-- Mlp Head

class Mlp(layers.Layer):

    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=layers.Activation(tf.nn.gelu), drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = TruncatedDense(hidden_features)
        self.act = act_layer
        self.fc2 = TruncatedDense(out_features)
        self.drop = layers.Dropout(drop)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'fc1': self.fc1,
            'act': self.act,
            'fc2': self.fc2,
            'drop': self.drop
          })
        return config

    def call(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x


#-- Model implementation : Hierarchical Transformer Autoencoder with Skip connections

def transformer_autoencoder(encoded_patches, embed_shape, input_resolution):
    
    encoded_list = []
    
    for iter1 in range(transformer_layers//2):

      if iter1 != 0:
        encoded_patches = PatchMerging(input_resolution, embed_shape[1])(encoded_patches)
        input_resolution //= 2       
        embed_shape = (input_resolution*input_resolution, embed_shape[1]*2)

      for _ in range(2):
        query = key = encoded_patches
        #---Multi-head Attention Layer
        attn_encoded_patches = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query, value=encoded_patches, key=key)
            
        attn_encoded_patches = layers.Dropout(rate)(attn_encoded_patches)
        #---Skip Connection 1
        encoded_patches += attn_encoded_patches
        #---Layer Normalization 1
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        #---Feed-Forward Network
        ffn_out = Mlp(in_features=embed_shape[1], hidden_features=4*embed_shape[1], drop=rate)(encoded_patches)
        #---Skip Connection 
        encoded_patches += ffn_out
        #---Layer Normalization 2
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
      
      encoded_list += [encoded_patches]
      
      
    for iter2 in range(transformer_layers//2):

      if iter2 != 0:
        
        encoded_patches = PatchExpanding(input_resolution, embed_shape[1])(encoded_patches)
        input_resolution *= 2       
        embed_shape = (input_resolution*input_resolution, embed_shape[1]//2)
        
        encoded_patches += encoded_list[3 - iter2]
        encoded_patches = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
      
      target = encoded_patches  
      
      for _ in range(2):
        
        query_tgt = key_tgt = target
            
        #---First Multi-Head Attention
        attn_target1 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=target, key=key_tgt) 
                                    
        attn_target1 = layers.Dropout(rate)(attn_target1)
        #---Skip Connection 1
        target += attn_target1
        #---Layer Normalization 1
        target = layers.LayerNormalization(epsilon=1e-6)(target)

        query_tgt = target
            
        #---Second Multi-Head Attention
        attn_target2 = layers.MultiHeadAttention(
            num_heads = num_heads,
            key_dim = embed_shape[1],
            dropout=rate)(query=query_tgt, value=encoded_patches, key=encoded_patches)
                                              
        attn_target2 = layers.Dropout(rate)(attn_target2)
        #---Skip Connection 2
        target += attn_target2
        #---Layer Normalization 2
        target = layers.LayerNormalization(epsilon=1e-6)(target)
        #---Feed-Forward Network
        ffn_out = Mlp(in_features=embed_shape[1], hidden_features=4*embed_shape[1], drop=rate)(encoded_patches)
        #---Skip Connection 3
        target +=ffn_out
        #---Layer Normalization 3
        target = layers.LayerNormalization(epsilon=1e-6)(target)
      
      encoded_patches =  target 

    return encoded_patches


#-- Build the H_TAE_S Model

def TAE():
   
  input_img = tf.keras.Input(shape=(image_size, image_size, num_channels))
  
  projection = layers.Conv2D(filters=filters, kernel_size=patch_size, strides=patch_size)(input_img)
  projection = layers.LayerNormalization(epsilon=1e-5)(projection) #-- si without
  B = tf.shape(projection)[0]
  enc_patches = tf.reshape(projection, [B, input_resolution*input_resolution, filters])
  decoded_patches = transformer_autoencoder(enc_patches, embed_shape, input_resolution) 
  decoded_patches = FinalPatchExpand_X4(input_resolution, filters)(decoded_patches) 
  B = tf.shape(decoded_patches)[0]
  decoded_patches = tf.reshape(decoded_patches, [B, image_size, image_size, filters]) 
  reconstructed = layers.Conv2D(filters=num_channels, kernel_size=1, use_bias=False)(decoded_patches)

  model = tf.keras.Model(input_img, reconstructed)
  
  return model
  
    
#-- Configure the hyperparameters

model_name = 'Hierarchical Transformer with Skip connections'
numEpochs = 50
learning_rate = 0.00001
rate = 0.  
image_size = 256
num_channels = 1
batch_size = 1


#-- Transformer parameters variation

param_grid = {
              'transformer_layers': [8],
              'patch_size' : [4],
              'num_heads' : [8],
              'filters' : [96]
             }
             
            
PARAMS = ParameterGrid(param_grid)
list_PARAMS = list(PARAMS)


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Train, Test and Evaluate

for param in list_PARAMS:
   try:

        #-- Checkpoints dir

        date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
        ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
        os.makedirs(ckpts_dir)

        ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
        params_path = os.path.join(ckpts_dir, 'Parameters.txt')
        results_path = os.path.join(ckpts_dir, 'Results.txt')
        fig_path = os.path.join(ckpts_dir, 'History_plot.png')
        dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
        predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
        residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
        residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')


        #-- Configure the parameters

        transformer_layers = param['transformer_layers']
        patch_size = param['patch_size']
        num_heads = param['num_heads']
        filters = param['filters']
        input_resolution = image_size // patch_size
        num_patches = input_resolution ** 2
        embed_shape = (num_patches, filters)


        #-- Configure the training

        opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)

        calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
        tqdm_callback = tfa.callbacks.TQDMProgressBar()

        model = TAE()
        model.summary()
        model.compile(optimizer=opt, loss='mse', metrics=['mae', SSIMLoss, MS_SSIMLoss])


        #-- Print & Write model Parameters

        parameters = (f'\nSelected model "{model_name}" with :\n - {num_heads}: Heads,\n - {patch_size}: Patch size,\n - {transformer_layers}: Transformer Layer,\n - ({embed_shape[0]}, {embed_shape[1]}): Embedding Shape,\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n')
        print(parameters)

        
        #-- Train

        print('\nTrain =>\n')
        history = model.fit(x = data_gen,
                            steps_per_epoch = training_steps,
                            validation_data = val_gen,
                            validation_steps = validation_steps,
                            verbose = 0,
                            epochs = numEpochs,
                            callbacks = [calbks, tqdm_callback]
                            )
        

        #-- Get training and test loss history

        plot_history(history, path=fig_path)
        plt.close()
        time.sleep(2)               
        
        
        #-- Test  

        print('\nTest ===>\n')
        my_test = np.load(test_path)
        brainmask = np.load(brainmask_path)
        x_prior = np.load(x_prior_path)
        my_labels = np.load(label_path)              
             
        healthy_test = np.load(test_healthy_path)
        steps = healthy_test.shape[0]
                
        score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])
        score_out = (f'\nTest on healthy unseen data :\n - Test loss (MSE): {score[0]},\n - Test MAE : {score[1]},\n - Test SSIM: {score[2]},\n - Test MS_SSIMLoss: {score[3]},\n')
        print(score_out)
        
               
        #-- Predict

        print('\nPredict =====>\n')
        predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
        np.save(predicted_path, predicted)
        time.sleep(4)


        #-- Calculate, Post-process and Save Residuals

        print('\nCalculate, Post-process and Save Residuals =====>\n')     
        residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_BP_path, residual_BP)
        
        residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
        np.save(residual_path, residual)
        

        #-- Evaluation

        print('\nEvaluate =========>\n')        
        [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
        results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
        print(results)
                     
        plt.figure()
        hor_axis = [x for x in range(len_testset)]
        plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
        plt.ylabel('Dice Score')
        plt.xlabel('N° Samples')
        plt.title('Dice scores')
        plt.savefig(dice_plot_path)
        time.sleep(2)


        #-- Save

        print('\nSave Results and Parameters =============>\n')
        f = open(results_path, "w")
        f.write(results)       
        f.close() 
                
        f = open(params_path, "w")
        f.write(parameters)
        f.write(score_out)
        f.close()
        
        
        #-- End

        print('\nEnd !\n')


   except:
        print('Error encountered !')
        tf.keras.backend.clear_session()
        continue